In [123]:
import requests
import json
import pandas as pd
import time
import os
from dotenv import load_dotenv

In [124]:
load_dotenv()

True

In [125]:
client_id = os.environ.get('CLIENT_ID')
client_secret = os.environ.get('CLIENT_SECRET')

In [126]:
def get_oauth_token(client_id = client_id, client_secret = client_secret):
    url = 'https://id.twitch.tv/oauth2/token'
    query_string = {'client_id': client_id, 
                    'client_secret': client_secret,
                    'grant_type': 'client_credentials'}
    r = requests.post(url, params=query_string).json()
    return r['access_token']

In [127]:
def get_top_100_categories(client_id = client_id):
    bearer_token = get_oauth_token()
    headers = {'Client-ID': client_id, 
               'Authorization': 'Bearer ' + bearer_token}
    url = r'https://api.twitch.tv/helix/games/top?first=100'
    r = requests.get(url, headers=headers)
    return r

In [128]:
def check_rate_limit_reached(req, ignore_limit = False):
    # checks if there is 1 request before rate limiting
    if int(req.headers['Ratelimit-Remaining']) <= 1:
        # not really preferred could cause trouble
        if ignore_limit:
            return int(req.headers['Ratelimit-Remaining'])
        print('Rate limit refreshes in 30s')
        time.sleep(30)
        print('Ready!')
    # return the remaining tries
    return int(req.headers['Ratelimit-Remaining'])

In [129]:
def get_top_100_streamers_for_each_game(games):
    streamers = {}
    
    bearer_token = get_oauth_token()
    headers = {'Client-ID': client_id, 
               'Authorization': 'Bearer ' + bearer_token}
    url = 'https://api.twitch.tv/helix/streams?first=100&game_id='
    
    for game in games['data']:
        req = requests.get(url + game['id'], headers=headers)
        check_rate_limit_reached(req)
        streamers[game['name']] = json.loads(req.text)
    return streamers

In [130]:
def json_to_df(json):
    total_streams_df = pd.DataFrame(
        columns = ['game_id','id','language','started_at','title','type','user_id','user_name','viewer_count'])
    for game_key in list(json.keys()):
        game_streams_df = pd.json_normalize(json[game_key]['data'])
        total_streams_df = pd.concat([total_streams_df, game_streams_df], sort = False)
    total_streams_df.drop(columns = ['thumbnail_url','tag_ids'], inplace = True)
    return total_streams_df

In [115]:
top_100_games = get_top_100_games().json()
top_streamers = get_top_100_streamers_for_each_game(top_100_games)

In [131]:
df = json_to_df(top_streamers)
df.head(10)

,game_id,id,language,started_at,title,type,user_id,user_name,viewer_count,user_login,game_name
0,509658,42293959406,en,2021-04-12T05:42:43Z,YUGIOH OH OH OH MOVIE NIGHT | !SUBATHON !GIFTC...,live,40934651,ludwig,30173,ludwig,Just Chatting
1,509658,42287932654,en,2021-04-11T18:30:55Z,💦🍑HOT TUB🍑💦shameless e-girl💦-- !instagram: am...,live,125387632,Amouranth,17818,amouranth,Just Chatting
2,509658,42294031710,en,2021-04-12T06:00:16Z,😍😍PRINCESS JACUZZI😍😍 1080P💓IG: indiefoxx.tv💓!s...,live,143917159,Indiefoxx,15206,indiefoxx,Just Chatting
3,509658,41742447996,fr,2021-04-12T08:40:41Z,PONCE - Culture & Plaisir !mdf !prog [20h30 : ...,live,50597026,Ponce,14172,ponce,Just Chatting
4,509658,41742384876,fr,2021-04-12T08:24:25Z,JV LE JOURNAL | lestream,live,147337432,lestream,6787,lestream,Just Chatting
5,509658,41399024077,ru,2021-04-12T07:11:48Z,СМОТРИМ 5 ЧАСОВ ОРЛА И РЕШКУ...СКИДКА НА ВОЗРА...,live,188890121,Dmitry_Lixxx,6530,dmitry_lixxx,Just Chatting
6,509658,41742798956,tr,2021-04-12T10:06:37Z,gelirim uğrarım ya | insta : erayozkenar,live,131403189,Eray,6038,eray,Just Chatting
7,509658,42294572846,ko,2021-04-12T08:23:32Z,비 싫어'-`,live,148057505,괴물쥐123,5756,tmxk319,Just Chatting
8,509658,41399732365,cs,2021-04-12T10:00:05Z,FeelsRainMan | FINÁLE TLOU,live,31453284,CzechCloud,3333,czechcloud,Just Chatting
9,509658,42293816062,zh,2021-04-12T05:11:05Z,🈲New【泰國😱鬼殺隊】👻【🎁送PS5 or Switch同捆任你選 - 參加輸入 !抽獎】...,live,25202416,蝦愛橘子,3265,shuteye_orange,Just Chatting


In [132]:
df.tail(10)

,game_id,id,language,started_at,title,type,user_id,user_name,viewer_count,user_login,game_name
87,75467,41742932636,fr,2021-04-12T10:36:23Z,En route pour Laventure (3700km) !discord #tea...,live,150539791,legeektv14,3,legeektv14,Euro Truck Simulator 2
88,75467,41399737725,de,2021-04-12T10:00:48Z,🔴 PC | ... gemütlich durch die Woche #TMP #Pro...,live,546743548,Schlesier77,3,schlesier77,Euro Truck Simulator 2
89,75467,41398661421,es,2021-04-12T05:40:08Z,Gaming in the morning =D Camionando,live,96307253,SlamFumOne,3,slamfumone,Euro Truck Simulator 2
90,75467,41742722348,fr,2021-04-12T09:49:45Z,"jour 12 defis km avec la G.L.E , Team L.G.E.F ...",live,189777019,sdin69,3,sdin69,Euro Truck Simulator 2
91,75467,41742818556,de,2021-04-12T10:11:07Z,"Gemütlich Fahren und plaudern, wer will kann m...",live,215061260,nusspanzer,3,nusspanzer,Euro Truck Simulator 2
92,75467,41742837628,it,2021-04-12T10:15:42Z,Continuiamo i viaggi per spagna/portogallo,live,233301185,gio_86best,3,gio_86best,Euro Truck Simulator 2
93,75467,41742726684,es,2021-04-12T09:50:47Z,🔴ETS.IBERIA RUTA POR EL NORTE DE ESPAÑA el cam...,live,128608678,xixo_g,2,xixo_g,Euro Truck Simulator 2
94,75467,41742815772,it,2021-04-12T10:10:27Z,viaggio di rutim...\n\n,live,646409106,pinnaxx76,2,pinnaxx76,Euro Truck Simulator 2
95,75467,41742532876,fr,2021-04-12T09:01:21Z,[PC/FR] Bonjour à tous et bonne journée,live,610740825,kenshireau7,2,kenshireau7,Euro Truck Simulator 2
96,75467,41742836556,tr,2021-04-12T10:15:27Z,ets vars hemen gels,live,612984982,fortwhite21,1,fortwhite21,Euro Truck Simulator 2


In [120]:
df.to_csv('./data/top_live_streamers.csv')